<a id='Capstone2: Predict which homes currently for sale would meet the requirements of a real estate investor'></a>

### 1.1 Capstone 2: Real Estate Investment Prediction

The purpose of this data science project is to come up with a model that predicts whether an available house may meet the requirements of a 'typical' real estate investor.

There are many homes entering the market, and they sell extremely quickly. It's difficult for an investor to calculate capitalization rate and look at the neighborhood, when most houses list mid-week and are sold by Sunday. One of the investors I spoke with has an intuitive sense of why he purchases in the particular neighborhoods and at a particular price-point, but he has to drive around to get a 'feel' for the property; when purchasing in a new city, he has to fly there and look at it in person. This isn't efficient. Which characteristics of a property truly mean most to an investor? Can we use data to tell the investor the source of his intuitive 'feel'?

This project aims to build a predictive model which will suggest which of the currently available homes would meet an investor's criteria, using both the inventory of the individual investor I spoke with, and the inventory of one of the largest home-rental companies in the United States, and school demographic data for schools with enrollment based on geographical boundaries. 

## 1.2 Imports & Keys

In [1]:
# Import packages
# Store the API keys and APP IDs as strings

import requests, json
import pandas as pd
from ast import literal_eval
import numpy as np

#create config.py file with your key, and add a .gitignore file with the name of the credential 
#file (i.e. config.py) so that your key isn't exposed to the world on GitHub if pushing there.
# import Configmel

SCHOOLDIGGER_APP_KEY = 'XXX'
SCHOOLDIGGER_API_KEY = 'XXX'

## 1.3 Corporate Investor Portfolio Data

### 1.3.1 Load the Corporate Investor Inventory Data

The first data to import is the inventory data from the large corporate investor. This company does not have an API, but the call their website uses to query their database was available through the developer console. I modified the call to include a suitable polygon of the Denver metro area (approximately the area banded by the beltway). 

In [2]:
# Get inventory data from the corporate investor's website using an HTTPS request taken fron the console 
# with some modifications to geographical area and return quantity
# Store the data as text.

corporateinvestor_url = "https://lease.invitationhomes.com/api/search-v2?sort=(by:distance,dir:asc)&center=(lat:39.739236,lng:-104.990251)&bounds=(east:-104.50630325447565,north:40.06708178814617,south:39.40982316328329,west:-105.47419874552435)&filters=()&amenities=()&offset=0&limit=1000&legacy=false"
r_inv = requests.get(corporateinvestor_url)
json_data_inv = r_inv.json()

In [3]:
# Print the response code and confirm the data type of the file created. 
print(r_inv.status_code, type(json_data_inv))

200 <class 'dict'>


In [4]:
# Create a dataframe from the inventory data using the relevant column names, and display the first 5 rows. 

corporate_inv = pd.DataFrame(json_data_inv['results'], columns=['property_code', 'address_1', 'city', 'state', 'zipcode', 'beds',
                                                'baths', 'sqft', 'market_rent', 'lat', 'lng'])
corporate_inv.head()

,property_code,address_1,city,state,zipcode,beds,baths,sqft,market_rent,lat,lng
0,10062936,4326 Wyandot St,Denver,CO,80211,3,2.00,1752,2850.00,39.77600000,-105.01437000
1,10049641,527 Leona Dr,Denver,CO,80221,3,3.00,1188,2375.00,39.84392000,-104.99356000
2,10062824,2361 Stacy Dr,Denver,CO,80221,3,2.00,1650,2310.00,39.84679000,-105.01484000
3,10096530,5120 Troy St,Denver,CO,80239,3,2.00,1864,2525.00,39.78991000,-104.84171000
4,10062679,9729 Detroit St,Thornton,CO,80229,4,2.00,2022,2550.00,39.87234000,-104.95237000


In [5]:
# How many properties are included currently in the corporate investor inventory available on the web? 
corporate_inv.shape

(51, 11)

Let's check the datatypes, and if necessary, clean them up. Here's what we expect, looking at the head of the dataframe:
'property_code', 'zipcode', 'beds', 'sqft': int64; 
'address_1', 'state': object; 
'baths', 'market_rent', 'lat', 'lng': float.

In [6]:
corporate_inv.dtypes

property_code    object
address_1        object
city             object
state            object
zipcode          object
beds              int64
baths            object
sqft              int64
market_rent      object
lat              object
lng              object
dtype: object

This is problematic because we cannot sort or manipulate by property code, for instance, if the property codes are sometimes strings and sometimes integers. Let's make the datatypes consistent. 

In [7]:
# Create a dictionary of datatypes then pass the string to the 'astype' method
data_types_dict = {'property_code': int, 'address_1': str, 'city': str, 'state': str, 'zipcode': int, 'beds': int, 
                  'baths': float, 'sqft': int, 'market_rent': float, 'lat': float, 'lng': float}
corporate_inv = corporate_inv.astype(data_types_dict)
corporate_inv.dtypes

property_code      int32
address_1         object
city              object
state             object
zipcode            int32
beds               int32
baths            float64
sqft               int32
market_rent      float64
lat              float64
lng              float64
dtype: object

The datatypes now look appropriate; we can move on to manipulating the data. 

### 1.3.2 Concatenate DataFrames and Check for Duplicates

There are approximately 60 records in our new dataframe, but the corporate investor probably owns far more. Due to the difficulty of collecting their portfolio holdings (properties are owned under multiple corporate names in multiple counties in this metro area), we will run this periodically and concatenate the new listings to the previously aquired listings, which have been saved locally, and check for duplicates. If done repeatedly, this list should grow over time. 

To summarize, we already have  cache of data. We need to load the previously collected data into a dataframe, concatenate the old/new dataframes, compare for duplicates, clean up the dataset (look for missing values), and write the resulting dataframe back to the csv. 

In [8]:
# Load the previously collected data from a csv into a new df, and print the head
corporate_old = pd.read_csv('../data/corporate_properties.csv')

In [9]:
# Combine the old and new inventory dfs and reset the index
inventory_combined = pd.concat([corporate_old, corporate_inv]).reset_index(drop=True)

In [10]:
#Print the head and shape of the combined df.
print(inventory_combined.head(), inventory_combined.shape)

   property_code           address_1           city state  zipcode  beds  \
0       10000753  10627 Kittredge St  Commerce City    CO    80022     3   
1       10000801   10705 Clermont St       Thornton    CO    80233     3   
2       10000898   10874 Fairfax Way       Thornton    CO    80233     4   
3       10000946  10944 Bellaire Way       Thornton    CO    80233     4   
4       10001199     11120 Sylvan Pl         Parker    CO    80138     3   

   baths  sqft  market_rent       lat        lng  
0    2.5  1993       2939.0  39.88982 -104.80151  
1    2.0  1544       2529.0  39.89058 -104.93351  
2    2.0  1904       2599.0  39.89348 -104.92838  
3    2.0  1800       2485.0  39.89548 -104.93625  
4    2.0  1387       2549.0  39.51228 -104.74424   (221, 11)


In [11]:
# Sort the values by the unique field 'property_code' and drop duplicates. 
inventory_combined.sort_values('property_code', inplace=True)
pd.DataFrame.drop_duplicates(inventory_combined, subset = ['property_code'], inplace=True)
inventory_combined.shape

(188, 11)

Count the number of missing values in each column and sort them.

In [12]:
#Count (using `.sum()`) the number of missing values (`.isnull()`) in each column of 
#combined_inventory as well as the percentages (using `.mean()` instead of `.sum()`).
#Order them (increasing or decreasing) using sort_values
#Call `pd.concat` to present these in a single table (DataFrame) with the helpful column names 'count' and '%'
missing = pd.concat([inventory_combined.isnull().sum(), round(100 * inventory_combined.isnull().mean(), 2)], axis=1)
missing.columns=["Count", "%"]
missing.sort_values('Count', inplace=True, ascending=False)
missing.head(15)

,Count,%
property_code,0,0.0
address_1,0,0.0
city,0,0.0
state,0,0.0
zipcode,0,0.0
beds,0,0.0
baths,0,0.0
sqft,0,0.0
market_rent,0,0.0
lat,0,0.0


There aren't any missing values in this dataset!

In [13]:
# If there are any missing values, remove the entire row (because data from Invitation should be complete
# unless there is an error)
inventory_combined.dropna(inplace=True)
inventory_combined.shape

(188, 11)

In [14]:
# Write the combined dataframe back to the csv. Overwrite the old file.
inventory_combined.to_csv('../data/corporate_properties.csv', index = False, header = True)

## 1.4 Individual Investor Data 

### 1.4.1 Load the Individual Investor Dataset

The second dataset to load into memory and examine is the .csv containing our individual investor's portfolio of local real estate investments. 

In [15]:
# Create a second dataframe from the inventory from private individual investor (obtained from the investor)
# Print the first 5 rows.
ind_inv = pd.read_csv('../raw_data/individualinvestor_properties.csv')
ind_inv = ind_inv[['Site Address', 'City', 'Mail State', 'Mail Zip', 'County', 'Status Notes', ]]
print(ind_inv.head(), ind_inv.shape)

              Site Address     City Mail State  Mail Zip   County Status Notes
0        3692     CHASE CT  Boulder         CO     80307  Boulder          NaN
1      424    S LASHLEY LN  Boulder         CO     80307  Boulder          NaN
2  3035     ONEAL PKWY 41S  Boulder         CO     80307  Boulder          NaN
3      285     SEMINOLE DR  Boulder         CO     80307  Boulder          NaN
4         840     FAITH CT  Boulder         CO     80303  Boulder          NaN (24, 6)


### 1.4.2 Examine and Clean the Data

I see that this data does not contain the square footage, current rent or lat/long location. This information may be difficult to put to use, unless I can find a way to automate the return of at least the square footage from the county assessor's site or another real estate repository. There are some unnecessary spaces between the street number and street name in the 'Site Address' column. Also, what is in the 'Status Notes' column? Let's look at missing data.

In [16]:
# Replace any number of spaces in 'Site Address' with one space
ind_inv['Site Address'] = ind_inv['Site Address'].str.replace('\s{2,}', ' ')
print(ind_inv.head(1))

    Site Address     City Mail State  Mail Zip   County Status Notes
0  3692 CHASE CT  Boulder         CO     80307  Boulder          NaN


<ipython-input-16-beec9552eb28>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  ind_inv['Site Address'] = ind_inv['Site Address'].str.replace('\s{2,}', ' ')


In [17]:
#Count (using `.sum()`) the number of missing values (`.isnull()`) in each column of 
#combined_inventory as well as the percentages (using `.mean()` instead of `.sum()`).
#Order them (increasing or decreasing) using sort_values
#Call `pd.concat` to present these in a single table (DataFrame) with the helpful column names 'count' and '%'
missing = pd.concat([ind_inv.isnull().sum(), round(100 * ind_inv.isnull().mean(), 2)], axis=1)
missing.columns=["Count", "%"]
missing.sort_values('Count', inplace=True, ascending=False)
missing.head(15)

,Count,%
Status Notes,16,66.67
Site Address,0,0.00
City,0,0.00
Mail State,0,0.00
Mail Zip,0,0.00
County,0,0.00


It looks like all of the missing data is within 'Status Notes'. 

In [18]:
# Look at all the unique values within the column, 'Status Notes'
ind_inv['Status Notes'].unique()

array([nan, 'Multi (3)', 'Multi', 'MH In Park',
       'Single Family Residential', 'Mobile Home Parks'], dtype=object)

It looks like 'Status Notes' contains information about the type of properties, including whether something is a multi-family property or a mobile home. The scope of this study is single family residences, only. We can delete all entries denoted as multi or mobile home. We will keep only rows with 'Status Notes' entry of 'Single Family Residential' or NaN. 

In [19]:
# Make a list of 'Status Notes' values that we will keep
status_options = ['Single Family Residential', np.nan] 
    
# selecting rows based on condition 
ind_inv = ind_inv[ind_inv['Status Notes'].isin(status_options)] 
ind_inv.shape

(17, 6)

There are 17 properties left in this dataset. It is possible that we will not use this dataset for the initial model. 

### 1.4.3 Save the Investor Data Locally

In [20]:
ind_inv.to_csv("../data/individualinvestor_properties.csv", index = False)

## 1.5 SchoolDigger API Data

Next, we can use the SchoolDigger API to get the school information and demographics. Per their documentation (developer.schooldigger.com), other characteristics of the school such as the percentage of students receiving free lunch, and the percentages of various ethnicities of the students, are also available. 

### 1.5.1 Load Data through API

Create the url as per the schooldigger documentation to include rankings from 2020, the shape of the denver metro area in question (the same as that called for the corporate investorproperties) and max per page, which is 50. We will not look at the private, virtual, charter and magnet schools. We will iterate the call to gather all of the data. 

First, we will call the first page of the response and check the JSON for the number of pages in the complete set. 

In [72]:
url = 'https://api.schooldigger.com/v1.2/schools?st=CO&boxLatitudeNW=40.06708178814617&boxLongitudeNW=-105.47419874552435&boxLatitudeSE=39.40982316328329&boxLongitudeSE=-104.50630325447565&perPage=50'   
url_params = {     # Parameters passed to the API
    "page": 1,  # We are going to iterate the page number
    "level": "Public",
    "isCharter": "False",
    "isVirtual": "False",
    "isMagnet": "False",
    "appID": "XXX",
    "appKey": "XXX"
    }
    
r = requests.get(url, params=url_params)

# 'schoolList' per API documentation contains needed data
# Create initial dataframe
firstpage = r.json()
df1 = pd.DataFrame.from_dict(firstpage['schoolList'])


In [133]:
# Get the total number of pages to be returned from the JSON
num_pages = firstpage['numberOfPages']
print(num_pages)

14


In [134]:
def get_schools(page_num): 
# This function launches the request for all school data in the Denver region.

# Parameters passed to the API
    url = 'https://api.schooldigger.com/v1.2/schools?st=CO&boxLatitudeNW=40.06708178814617&boxLongitudeNW=-105.47419874552435&boxLatitudeSE=39.40982316328329&boxLongitudeSE=-104.50630325447565&perPage=50'   
    url_params = {     
    "page": page_num,  
    "level": "Public",
    "isCharter": "False",
    "isVirtual": "False",
    "isMagnet": "False",
    "appID": "XXX",
    "appKEY": "XXX"    
    }

    
    response = requests.get(url, params=url_params)
    return response.json()

# Iterate over the page_num up to the max number of pages. Call the get_schools function and append result to the dataframe. 
for page_num in range(2,(num_pages + 1)) : 
    try:
            output_json = get_schools(page_num) 
            df2 = pd.DataFrame.from_dict(output_json['schoolList'])
            df1 = df1.append(df2)

    except AttributeError:
            print("error at ", page_num)
            

In [135]:
# Save the raw file to local computer. Altered the keys so that I cannot accidentally make the call again (paid API)
df1.to_csv("../raw_data/school_digger.csv", index = False)

In [40]:
# Load the dataframe back from the local file, for testing without running above code.  
schooldigger_raw_df = pd.read_csv("../raw_data/school_digger.csv")

In [41]:
# Somehow, my raw data file ended up with duplicates:
schooldigger_raw_df.sort_values(by='schoolid').head()

,schoolid,schoolName,address,lowGrade,highGrade,schoolLevel,district,county,rankMovement,schoolYearlyDetails,hasBoundary
508,80000400709,Rocky Mountain School Of Expeditionary Learning,"{'latLong': {'latitude': 39.6854, 'longitude':...",K,12,Alternative,"{'districtID': '0800004', 'districtName': 'Den...","{'countyName': 'Denver County', 'countyURL': '...",NaN,"[{'year': 2020, 'numberOfStudents': 368, 'perc...",False
359,80028606668,Lookout Mountain,"{'latLong': {'latitude': 39.741558, 'longitude...",5,12,High,"{'districtID': '0800286', 'districtName': 'Div...","{'countyName': 'Jefferson County', 'countyURL'...",NaN,"[{'year': 2020, 'numberOfStudents': 0, 'percen...",False
417,80028606703,Mount View Youth Services Center,"{'latLong': {'latitude': 39.647876, 'longitude...",5,12,High,"{'districtID': '0800286', 'districtName': 'Div...","{'countyName': 'Jefferson County', 'countyURL'...",NaN,"[{'year': 2020, 'numberOfStudents': 0, 'percen...",False
49,80028606742,Betty K Marler Youth Services Center,"{'latLong': {'latitude': 39.647876, 'longitude...",6,12,High,"{'districtID': '0800286', 'districtName': 'Div...","{'countyName': 'Jefferson County', 'countyURL'...",NaN,"[{'year': 2020, 'numberOfStudents': 0, 'percen...",False
5,80195000009,Adams City Middle School,"{'latLong': {'latitude': 39.8282, 'longitude':...",6,8,Middle,"{'districtID': '0801950', 'districtName': 'Sch...","{'countyName': 'Adams County', 'countyURL': 'h...",-38.0,"[{'year': 2020, 'numberOfStudents': 773, 'perc...",True


In [42]:
# Drop duplicates by school id
schooldigger_raw_df.drop_duplicates(subset="schoolid", inplace=True)

In [43]:
# If there are no duplicates, the above code will not make any changes. 
schooldigger_raw_df.to_csv("../raw_data/school_digger.csv", index = False)

### 1.5.2 Narrow Down the Dataset for Desired Fields

In [3]:
# List the column names of the data
print(schooldigger_raw_df.columns)

Index(['schoolid', 'schoolName', 'phone', 'url', 'urlCompare', 'address',
       'lowGrade', 'highGrade', 'schoolLevel', 'isCharterSchool',
       'isMagnetSchool', 'isVirtualSchool', 'isTitleISchool',
       'isTitleISchoolwideSchool', 'district', 'county', 'rankHistory',
       'rankMovement', 'schoolYearlyDetails', 'isPrivate', 'privateDays',
       'privateHours', 'privateHasLibrary', 'privateCoed',
       'privateOrientation', 'hasBoundary'],
      dtype='object')


Because we did not include any virtual schools, magnet schools, charter schools, or private schools in the call to the API, we can safely delete those columns of the dataframe. We also do not need the url, urlcompare, the phone number, and some other superfluous columns for this project. 

In [4]:
# Make a list of columns to keep and then keep just those columns. 
# Print out the new list of columns to confirm changes.
keep_columns = ['schoolid', 'schoolName', 'address', 'lowGrade', 'highGrade', 'schoolLevel', 'district', 'county',
                'rankMovement', 'schoolYearlyDetails', 'hasBoundary']
schooldigger_raw_df = schooldigger_raw_df[keep_columns]
print(schooldigger_raw_df.columns)

Index(['schoolid', 'schoolName', 'address', 'lowGrade', 'highGrade',
       'schoolLevel', 'district', 'county', 'rankMovement',
       'schoolYearlyDetails', 'hasBoundary'],
      dtype='object')


### 1.5.3 Correct Data Types, and Unnest Lists and Dictionaries within the SchoolDigger set

Some nice data is under 'schoolYearlyDetails' as a nested list, and under 'address', 'district' and 'county' as dictionaries. 

In [5]:
# Display the type of data in columns
list_strings = ['schoolYearlyDetails','address', 'county', 'district']
for column in list_strings:    
    print(type(schooldigger_raw_df[column][0]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


These fields are currently strings, so first we need to change them dict / list. 

In [6]:
# Use literal_eval to change strings to list/dict and print out the current data type of those series. 
for column in list_strings:
    schooldigger_raw_df[column] = schooldigger_raw_df[column].apply(lambda x: literal_eval(str(x)))
for column in list_strings:    
    print(type(schooldigger_raw_df[column][0]))

<class 'list'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


We can use the pandas json_normalize function to flatten the dictionaries within the 'address', 'county', and 'district' columns. 

In [7]:
# Use panda's json_normalize to flatten the dictionaries within the columns.
df_address = pd.json_normalize(schooldigger_raw_df['address'])
df_county = pd.json_normalize(schooldigger_raw_df['county'])
df_district = pd.json_normalize(schooldigger_raw_df['district'])

In [8]:
# Reset all of the indexes
df_address.reset_index(drop=True, inplace=True)
df_county.reset_index(drop=True, inplace=True)
df_district.reset_index(drop=True, inplace=True)
schooldigger_raw_df.reset_index(drop=True, inplace=True)

In [9]:
# Look at the data in the 'county' df. We will only keep the 'countyName' column, because the other column contains
# URLs back to the API website which generate 404 errors. 
print(df_county.columns)
print(df_county.head())

Index(['countyName', 'countyURL'], dtype='object')
       countyName                                          countyURL
0   Denver County  https://www.schooldigger.com/go/CO/county/Denv...
1   Denver County  https://www.schooldigger.com/go/CO/county/Denv...
2    Adams County  https://www.schooldigger.com/go/CO/county/Adam...
3  Douglas County  https://www.schooldigger.com/go/CO/county/Doug...
4    Adams County  https://www.schooldigger.com/go/CO/county/Adam...


In [10]:
# look at the data in the 'district' df. We will only keep the 'districtName', because again the other columns contain
# URLs, and we will not be using the 'districtID'
print(df_district.columns)

Index(['districtID', 'districtName', 'url', 'rankURL'], dtype='object')


In [11]:
# Look at the data in the 'address' df. We will keep the street, city, state, zip, latitude and longitude.
# We will drop the full state name, the zip+4, and the URLs back to the API source. 
print(df_address.columns)

Index(['street', 'city', 'state', 'stateFull', 'zip', 'zip4', 'cityURL',
       'zipURL', 'html', 'latLong.latitude', 'latLong.longitude'],
      dtype='object')


In [12]:
# Now concatentate the normalized columns to the main df, and drop all of the unwanted columns as discussed above. 
schooldigger_explode_df = pd.concat([schooldigger_raw_df, df_address, df_county, df_district], axis=1)
schooldigger_explode_df.drop(['address', 'stateFull', 'cityURL', 'zipURL', 'html', 'countyURL', 'url', 'rankURL', 
                         'district', 'districtID', 'county'], axis = 1, inplace=True)

In [13]:
#schooldigger_explode_df.reset_index(drop=True, inplace=True)
print(schooldigger_explode_df.head(2))

      schoolid                   schoolName lowGrade highGrade schoolLevel  \
0  80336000305  Abraham Lincoln High School        9        12        High   
1  80336001750  Academia Ana Marie Sandoval       PK         6  Elementary   

   rankMovement                                schoolYearlyDetails  \
0         -12.0  [{'year': 2020, 'numberOfStudents': 971, 'perc...   
1         -13.0  [{'year': 2020, 'numberOfStudents': 423, 'perc...   

   hasBoundary                        street    city state    zip  zip4  \
0         True  2285 South Federal Boulevard  Denver    CO  80219  5433   
1         True           3655 Wyandot Street  Denver    CO  80211  2950   

   latLong.latitude  latLong.longitude     countyName           districtName  
0           39.6761          -105.0265  Denver County  School District No. 1  
1           39.7675          -105.0155  Denver County  School District No. 1  


In [14]:
# Now let's look at the 'schoolYearlyDetails' which contains a dictionary, by printing one entry to the console to examine 
print(schooldigger_explode_df['schoolYearlyDetails'][0])

[{'year': 2020, 'numberOfStudents': 971, 'percentFreeDiscLunch': 90.73, 'percentofAfricanAmericanStudents': 4.02, 'percentofAsianStudents': 2.88, 'percentofHispanicStudents': 88.36, 'percentofIndianStudents': 1.24, 'percentofPacificIslanderStudents': 0.0, 'percentofWhiteStudents': 2.99, 'percentofTwoOrMoreRaceStudents': 0.51, 'percentofUnspecifiedRaceStudents': None, 'teachersFulltime': 65.0, 'pupilTeacherRatio': 14.9, 'numberofAfricanAmericanStudents': 39, 'numberofAsianStudents': 28, 'numberofHispanicStudents': 858, 'numberofIndianStudents': 12, 'numberofPacificIslanderStudents': 0, 'numberofWhiteStudents': 29, 'numberofTwoOrMoreRaceStudents': 5, 'numberofUnspecifiedRaceStudents': None}, {'year': 2019, 'numberOfStudents': 952, 'percentFreeDiscLunch': 91.07, 'percentofAfricanAmericanStudents': 3.05, 'percentofAsianStudents': 3.99, 'percentofHispanicStudents': 88.55, 'percentofIndianStudents': 1.05, 'percentofPacificIslanderStudents': 0.0, 'percentofWhiteStudents': 2.63, 'percentofTwoO

The column 'schoolYearlyDetails' is a list of dictionaries. We are going to handle this by first moving it to a series, then dropping the '2019' data. We will drop the '2019' data because we will examine the potential investment properties with their present characteristics, and not look at changes over time. 

In [15]:
# Turn the column into a series; and drop the data from 2019. Drop the column from the main schooldigger df which was 
# made into a series. 
details_series = schooldigger_explode_df['schoolYearlyDetails'].apply(pd.Series).reset_index(drop=True)
details_series.drop([1], axis=1, inplace=True)
schooldigger_explode_df.drop(['schoolYearlyDetails'], axis = 1, inplace=True)

In [16]:
schooldigger_explode_df.columns

Index(['schoolid', 'schoolName', 'lowGrade', 'highGrade', 'schoolLevel',
       'rankMovement', 'hasBoundary', 'street', 'city', 'state', 'zip', 'zip4',
       'latLong.latitude', 'latLong.longitude', 'countyName', 'districtName'],
      dtype='object')

In [17]:
# Reset the index of the schoolYearlyDetails series and print the head to the console
details_series.reset_index(drop=True, inplace=True)
print(details_series.head(2), details_series.shape)

                                                   0
0  {'year': 2020, 'numberOfStudents': 971, 'perce...
1  {'year': 2020, 'numberOfStudents': 423, 'perce... (1292, 1)


In [18]:
# Concat the main schooldigger df with this details series.
schooldigger_explode_df = pd.concat([schooldigger_explode_df, details_series], axis=1)

In [19]:
# Rename a few of the columns to make it easier to visually parse
schooldigger_explode_df.rename(columns = {0:'Details', 'latLong.latitude':'latitude', 'latLong.longitude':'longitude'},
                               inplace = True)

In [20]:
# create new df of the Details column and use apply(pd.Series), print the column name to pick which we want to keep.
df_deets = schooldigger_explode_df['Details'].apply(pd.Series)
print(df_deets.columns)

Index([                                 0,                 'numberOfStudents',
        'numberofAfricanAmericanStudents',            'numberofAsianStudents',
               'numberofHispanicStudents',           'numberofIndianStudents',
        'numberofPacificIslanderStudents',    'numberofTwoOrMoreRaceStudents',
        'numberofUnspecifiedRaceStudents',            'numberofWhiteStudents',
                   'percentFreeDiscLunch', 'percentofAfricanAmericanStudents',
                 'percentofAsianStudents',        'percentofHispanicStudents',
                'percentofIndianStudents', 'percentofPacificIslanderStudents',
         'percentofTwoOrMoreRaceStudents', 'percentofUnspecifiedRaceStudents',
                 'percentofWhiteStudents',                'pupilTeacherRatio',
                       'teachersFulltime',                             'year'],
      dtype='object')


In [21]:
# Filter the df by the list of columns we wish to keep; Because we are keeping all the % columns and the total column, 
# we can drop the columns containing number (because % * total will give us back number)
list_deets = ['numberOfStudents', 'percentFreeDiscLunch', 'percentofAfricanAmericanStudents', 
                    'percentofAsianStudents', 'percentofHispanicStudents', 'percentofWhiteStudents', 'percentofIndianStudents',
                    'percentofPacificIslanderStudents','percentofTwoOrMoreRaceStudents', 'percentofUnspecifiedRaceStudents',
                    'pupilTeacherRatio', 'year']
df_deets = df_deets[list_deets].reset_index()

In [22]:
# Create 'clean' schooldigger df by concatenating the main schooldigger df and the df_deets column-wise. Drop the
# un-expanded column, 'Details' because it contains information already contained in the df. 
schooldigger_clean = pd.concat([schooldigger_explode_df, df_deets], axis=1)
schooldigger_clean.drop(['Details'], axis = 1, inplace=True)

In [23]:
schooldigger_clean.columns

Index(['schoolid', 'schoolName', 'lowGrade', 'highGrade', 'schoolLevel',
       'rankMovement', 'hasBoundary', 'street', 'city', 'state', 'zip', 'zip4',
       'latitude', 'longitude', 'countyName', 'districtName', 'index',
       'numberOfStudents', 'percentFreeDiscLunch',
       'percentofAfricanAmericanStudents', 'percentofAsianStudents',
       'percentofHispanicStudents', 'percentofWhiteStudents',
       'percentofIndianStudents', 'percentofPacificIslanderStudents',
       'percentofTwoOrMoreRaceStudents', 'percentofUnspecifiedRaceStudents',
       'pupilTeacherRatio', 'year'],
      dtype='object')

### 1.5.4 Explore the Cleaned SchoolDigger Data

Now that the data is orderly, we can look at the values within the dataset. It's pretty large at 671 schools, which dwarfs the quantity of the records we have for the investment portfolios. Let's look at some of the data, making sure that it's relevant for our model, which again, is to look at the feeder schools for particular geographical regions. 

Let's start by looking at the 'hasBoundary' column. 

In [24]:
# Explore some of the data. What is in the 'hasBoundary' column?
print(schooldigger_clean['hasBoundary'].unique())
(schooldigger_clean['hasBoundary'].values == True).sum()

[ True False]


1125

We are interested in schools where the majority of their students are pulled from a boundary area, because if the school draws students from a wide area, it doesn't reflect the characteristics of the neighborhood. It looks like the overwhelming majority of schools in our dataset have boundaried enrollment areas. What does the data look like where 'hasBoundary' is False?

In [69]:
# Filter the schooldigger_clean df for rows where 'hasBoundary' == False 
df = schooldigger_clean[(schooldigger_clean['hasBoundary'].values == False)]
print(df.head(25))

        schoolid                                         schoolName lowGrade  \
7    80258006343                         Adams Youth Service Center        4   
8    80690006435                        Adams12 Five Star Preschool       PK   
14   80291006750                         Altitude Elementary School       PK   
18   80234006625                 Aps Early Beginnings - A Zoom Site       PK   
21   80249001219                         Arapahoe Ridge High School        9   
43   80336006682                   Bear Valley International School        6   
49   80028606742               Betty K Marler Youth Services Center        6   
50   80555006671             Big Picture College And Career Academy        9   
61   80480001907                           Brady Exploration School        9   
75   80336000322              Career Education Center Early College        9   
114  80378001310            Colorado's Finest High School Of Choice        9   
125  80480006306  Connections Learning C

It looks like some of these are preschools, and some appear to be alternative high schools. If a record describes a preschool or an alternative highschool, there is no rank movement *and* the hasBoundary is false. Let's remove the data where 'rankMovement' is NaN, and see how many records of schools without an enrollment boundary remain. 

In [70]:
# Filter the schooldigger_clean df on those rows where 'rankMovement' is NaN, and sum the number of 'hasBoundary'==False
schooldigger_clean = schooldigger_clean[~schooldigger_clean['rankMovement'].isnull()]
(schooldigger_clean['hasBoundary'].values == False).sum()

29

That removed all but 15 of the rows where the 'hasBoundary' is false. Let's look at those remaining rows:

In [71]:
# Filter the schooldigger_clean df on 'hasBoundary' == False 
schooldigger_clean[(schooldigger_clean['hasBoundary'].values == False)]

,schoolid,schoolName,lowGrade,highGrade,schoolLevel,rankMovement,hasBoundary,street,city,state,...,percentofAfricanAmericanStudents,percentofAsianStudents,percentofHispanicStudents,percentofWhiteStudents,percentofIndianStudents,percentofPacificIslanderStudents,percentofTwoOrMoreRaceStudents,percentofUnspecifiedRaceStudents,pupilTeacherRatio,year
21,80249001219,Arapahoe Ridge High School,9,12,High,-39.0,False,6600 Arapahoe Avenue,Boulder,CO,...,1.33,2.00,59.33,33.33,2.00,0.00,2.00,NaN,6.2,2020.0
75,80336000322,Career Education Center Early College,9,12,High,5.0,False,2650 Eliot Street,Denver,CO,...,0.81,1.02,94.09,3.26,0.20,0.00,0.61,NaN,12.4,2020.0
114,80378001310,Colorado's Finest High School Of Choice,9,12,High,-21.0,False,300 W Chenango,Englewood,CO,...,2.01,0.00,33.78,61.20,0.33,0.00,2.68,NaN,14.7,2020.0
126,80336000380,Contemporary Learning Academy,9,12,High,-19.0,False,200 E. 9th Ave.,Denver,CO,...,41.13,0.81,36.29,13.71,2.42,0.00,5.65,NaN,8.4,2020.0
176,80345001527,Eagle Academy,9,12,High,7.0,False,9375 Cresthill Lane,Highlands Ranch,CO,...,2.21,5.15,18.38,67.65,0.74,0.00,5.88,NaN,10.6,2020.0
200,80336000345,Emily Griffith High School,9,12,High,-6.0,False,1860 Lincoln St.,Denver,CO,...,20.00,1.56,58.70,15.58,0.78,0.52,2.86,NaN,20.2,2020.0
227,80336001575,Florence Crittenton High School,9,12,High,21.0,False,55 South Zuni,Denver,CO,...,8.57,0.95,84.76,4.76,0.00,0.00,0.95,NaN,9.1,2020.0
238,80480001920,Free Horizon Montessori,PK,8,Elementary,51.0,False,15920 W 10th Ave,Golden,CO,...,0.66,1.53,9.19,83.37,0.00,0.00,5.25,NaN,18.8,2020.0
298,80258000729,Innovations & Options,6,12,High,0.0,False,830 Bridge Street,Brighton,CO,...,1.37,1.02,57.34,36.52,0.68,0.00,3.07,NaN,22.5,2020.0
351,80195001307,Lester R Arnold High School,9,12,High,-14.0,False,6500 East 72nd Avenue,Commerce City,CO,...,1.98,0.00,83.40,13.04,0.79,0.00,0.79,NaN,20.0,2020.0


It looks like these are either Montessori, or alternative-style high schools. We may need to just remove all of these, because they aren't regular feeder schools for the areas, but maybe their presence has some effect on the data. We should leave them in for now. 

We haven't noticed any missing data of significance in this dataset, but let's check to be sure. 

In [72]:
#Count (using `.sum()`) the number of missing values (`.isnull()`) in each column of 
#combined_inventory as well as the percentages (using `.mean()` instead of `.sum()`).
#Order them (increasing or decreasing) using sort_values
#Call `pd.concat` to present these in a single table (DataFrame) with the helpful column names 'count' and '%'
missing = pd.concat([schooldigger_clean.isnull().sum(), round(100 * schooldigger_clean.isnull().mean(), 2)], axis=1)
missing.columns=["Count", "%"]
missing.sort_values('Count', inplace=True, ascending=False)
missing.head(15)

,Count,%
percentofUnspecifiedRaceStudents,1098,100.00
pupilTeacherRatio,2,0.18
percentofTwoOrMoreRaceStudents,2,0.18
percentofPacificIslanderStudents,2,0.18
percentofIndianStudents,2,0.18
percentofWhiteStudents,2,0.18
percentofHispanicStudents,2,0.18
percentofAsianStudents,2,0.18
percentofAfricanAmericanStudents,2,0.18
percentFreeDiscLunch,2,0.18


There isn't any data at all in the 'percentofUnspecifiedRaceStudents' column. That column should be removed since it doesn't offer any information. Also, It looks like there may be one record that's missing several fields. Let's look for it:

In [73]:
# Filter schooldigger_clean against the blank column
schooldigger_clean = schooldigger_clean.loc[:, schooldigger_clean.columns != 'percentofUnspecifiedRaceStudents']

In [74]:
# Filter schooldigger_clean for the missing records
schooldigger_clean[schooldigger_clean['pupilTeacherRatio'].isnull()]

,schoolid,schoolName,lowGrade,highGrade,schoolLevel,rankMovement,hasBoundary,street,city,state,...,percentFreeDiscLunch,percentofAfricanAmericanStudents,percentofAsianStudents,percentofHispanicStudents,percentofWhiteStudents,percentofIndianStudents,percentofPacificIslanderStudents,percentofTwoOrMoreRaceStudents,pupilTeacherRatio,year
619,80555000959,Valley View K-8,K,8,Elementary,45.0,True,660 West 70th Avenue,Denver,CO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.0
1240,80555000959,Valley View K-8,K,8,Elementary,45.0,True,660 West 70th Avenue,Denver,CO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.0


In [75]:
schooldigger_clean.columns

Index(['schoolid', 'schoolName', 'lowGrade', 'highGrade', 'schoolLevel',
       'rankMovement', 'hasBoundary', 'street', 'city', 'state', 'zip', 'zip4',
       'latitude', 'longitude', 'countyName', 'districtName', 'index',
       'numberOfStudents', 'percentFreeDiscLunch',
       'percentofAfricanAmericanStudents', 'percentofAsianStudents',
       'percentofHispanicStudents', 'percentofWhiteStudents',
       'percentofIndianStudents', 'percentofPacificIslanderStudents',
       'percentofTwoOrMoreRaceStudents', 'pupilTeacherRatio', 'year'],
      dtype='object')

There is one record which is missing all the information taken from 'schoolYearlyDetails'. We will leave this record in the dataframe because 'hasBoundary' = True and it has a rankMovement. 

### 1.5.5 Save the SchoolDigger Data Locally

In [114]:
# Send the df as a csv to the local drive
schooldigger_clean.to_csv("../data/school_digger.csv", index = False)

## 1.6 Get Current Listing Data - Homes For Sale

### 1.6.1 Load Current Listings from Website

A website that accumulates listings from other MLS-sponsored webpages is used here. It lists a variety of data about each home for sale, because it's sourced from several other websites. Because of this, it's also pretty straightforward to reuse a query from their website. Let's start by importing a file with an HTTP request.

In [15]:
# url of query, using min price of 200k, max of 750k, and the same lat-long shape as used in other queries for this project
url_denverhomes = "https://www.searchdenverhomes.com/api/v2/search/?page=1&list_price_max=750000&neighborhood=all&baths_min=all&beds_min=all&type=res&list_price_min=200000&&per_page=1000&latitude_min=39.609319&longitude_min=-105.248336&latitude_max=39.844849&longitude_max=-104.725113"
# Get request and response
r = requests.get(url_denverhomes)
json_data = r.json()
print(r.status_code, type(json_data))

200 <class 'list'>


In [16]:
# Create df from json_data, look at sample record entry
listings_df = pd.DataFrame(json_data, columns=['fields'])
print(listings_df.head(), type(listings_df['fields'][0]), listings_df.shape)

                                              fields
0  {'living_area_above_ground': {'data': 914, 'la...
1  {'living_area_above_ground': {'data': 1615, 'l...
2  {'living_area_above_ground': {'data': 1583, 'l...
3  {'living_area_above_ground': {'data': 1104, 'l...
4  {'living_area_above_ground': {'data': 1745, 'l... <class 'dict'> (697, 1)


### 1.6.2 Flatten the Data

All of the listing data is in one column, and the fields consist of a very long dictionary. Let's use json_normalize to flatten the data, separating it into columns that are readable and parsable. 

In [17]:
# Because the df consists of dict, use pd.normalize and look at columns
listings_df = pd.json_normalize(listings_df['fields'])

In [18]:
# Make a list of all the column names, because there are a lot
my_list = listings_df.columns.values.tolist()
print(my_list)

['living_area_above_ground.data', 'living_area_above_ground.label', 'list_price.raw', 'list_price.data', 'list_price.label', 'school_district.data', 'school_district.label', 'zoned.data', 'zoned.label', 'total_carport_spaces.data', 'total_carport_spaces.label', 'total_garage_spaces.data', 'total_garage_spaces.label', 'living_area.data', 'living_area.label', 'lot_size.data', 'lot_size.label', 'taxes.data', 'taxes.label', 'baths.data', 'baths.label', 'images.data', 'neighborhood.data', 'neighborhood.label', 'listing_office.data', 'listing_office.label', 'property_id.data', 'city.data', 'city.label', 'style.data', 'style.label', 'mls_number.data', 'mls_number.label', 'zip.data', 'zip.label', 'basement_finished.data', 'basement_finished.label', 'sold_price.data', 'sold_price.label', 'acres.data', 'acres.label', 'state.data', 'state.label', 'sewer.data', 'sewer.label', 'elementary_school.data', 'elementary_school.label', 'contact_url.data.url', 'contact_url.data.show_on_details', 'latitude.

### 1.6.3 Narrow Down the Data: What are we looking for?

There's a lot of extra data here which we aren't going to include in our model for possible future rental homes. For example, we don't need any of the columns which keep record-keeping data, such as labels, from the source website. We don't need any contact information, disclaimers, or URLs. We do require information about the physical characteristics of each property, similar in scope to the data we have already collected from the investment profile, and we need any information which contributes to the cost of ownership (list price, HOA if any, and annual taxes). 

In [19]:
# Filter the df for just the columns we want
listings_df = listings_df[['list_price.raw', 'school_district.data','taxes.data',
                          'baths.data', 'city.data', 'zip.data','elementary_school.data', 'latitude.data', 
                          'square_footage.data','middle_school.data',
                          'high_school.data', 'longitude.data','beds.data', 'street.data', 'hoa_fees.data',
                          'hoa_frequency.data']]

In [20]:
# Rename a few of the columns to make it easier to visually parse
listings_df.rename(columns = {'list_price.raw':'listPrice', 'school_district.data':'district', 'living_area.data':'livingArea',
                              'taxes.data':'taxes', 'baths.data':'baths', 'city.data':'city', 'zip.data':'zip',
                              'elementary_school.data':'elementarySchool', 'latitude.data':'lat', 'longitude.data':'lng', 
                              'square_footage.data':'sqft', 'middle_school.data':'middleSchool', 'high_school.data':'highSchool',
                              'beds.data':'beds', 'street.data':'address', 'hoa_fees.data':'hoaFees', 'hoa_frequency.data':'hoaFreq'},
                               inplace = True)
print(listings_df.head())

   listPrice              district   taxes  baths         city    zip  \
0     750000  Jefferson County R-1  $1,664    2.0  Wheat Ridge  80214   
1     750000              Denver 1  $2,749    3.0       Denver  80210   
2     750000              Denver 1  $2,205    2.0       Denver  80205   
3     750000              Denver 1  $2,354    2.0       Denver  80207   
4     750000        Cherry Creek 5  $3,754    3.0       Denver  80222   

                elementarySchool        lat   sqft      middleSchool  \
0                        Lumberg  39.759313  1,751         Jefferson   
1                         Asbury  39.687801  1,615             Grant   
2  Cole Arts And Science Academy  39.762607  1,583             Wyatt   
3                        Stedman  39.755537  1,104  Bill Roberts E-8   
4                    Holly Hills  39.659757  3,099              West   

     highSchool         lng  beds               address hoaFees hoaFreq  
0     Jefferson -105.055104   4.0    2920 Benton Stree

In [21]:
# Reorder the columns to make more sense
listings_ordered = listings_df.loc[:, ['address', 'city', 'zip', 'beds', 'baths', 'sqft', 'listPrice', 'taxes', 
                                       'hoaFees', 'hoaFreq','lat', 'lng', 'district', 'elementarySchool', 'middleSchool',
                                      'highSchool']]

### 1.6.4 Explore the Cleaned For-Sale Listings

We can see from the small snippet of code that there are missing values in the hoaFees column. Let's look at those unique values. 

In [22]:
# Show the unique values present in the 'hoaFees column'
listings_ordered['hoaFees'].unique()

array([nan, '$43', '$0', '$113', '$65', '$700', '$400', '$150', '$350',
       '$35', '$80', '$63', '$500', '$68', '$79', '$250', '$1,250',
       '$357', '$49', '$195', '$440', '$145', '$55', '$30', '$375',
       '$180', '$185', '$85', '$33', '$270', '$388', '$39', '$795',
       '$183', '$140', '$302', '$310', '$62', '$59', '$42', '$498',
       '$130', '$100', '$58', '$353', '$200', '$67', '$132', '$70',
       '$295', '$122', '$373', '$374', '$45', '$330', '$235', '$129',
       '$660', '$99', '$98', '$44', '$160', '$2,700'], dtype=object)

Let's change all the NaN to $0 so that the data is consistent.

In [23]:
# Use fillna to replace NaN with '$0'
listings_ordered['hoaFees'].fillna('$0',inplace=True)

We need to be able to calculate a capitalization rate on proposed properties. There should be a value in the 'hoaFees' column where there is a value in the 'hoaFreq' column. Let's look at the unique values in the 'hoaFreq' column:

In [24]:
# how many values in 'hoaFreq' are not NaN?
listings_ordered['hoaFreq'].count(), listings_ordered['hoaFreq'].unique()

(119,
 array([nan, 'Monthly', 'Annually', 'Semi-Annually', 'Quarterly'],
       dtype=object))

In [25]:
# fill Nan in 'hoaFreq' with 'N/A'
listings_ordered['hoaFreq'].fillna('N/A', inplace=True)

In [26]:
# Are there any cases where there is an hoaFee, but no corresponding frequency value?
listings_ordered[~listings_ordered['hoaFees'].isin(['$0']) & (listings_ordered['hoaFreq'] == 'N/A')]

,address,city,zip,beds,baths,sqft,listPrice,taxes,hoaFees,hoaFreq,lat,lng,district,elementarySchool,middleSchool,highSchool
343,5575 Argonne Court,Denver,80249,3.0,3.0,"2,572",550000,"$4,377",$150,N/A,39.797259,-104.769374,Denver 1,Waller,Dr. Martin Luther King,Dr. Martin Luther King


Using Zillow, we see that the listing here has an HOA which is included already in the taxes, so we can change that value to 0.

In [27]:
listings_ordered.at[343,'hoaFees']='$0'
listings_ordered[~listings_ordered['hoaFees'].isin(['$0']) & (listings_ordered['hoaFreq'] == 'N/A')]

,address,city,zip,beds,baths,sqft,listPrice,taxes,hoaFees,hoaFreq,lat,lng,district,elementarySchool,middleSchool,highSchool


Every instance of an HOA fee over $0 has a frequency, so we will be able to calculate the capitalization. 
Now that we have the HOA data cleaned up, let's look at missing values. 

In [28]:
#Count (using `.sum()`) the number of missing values (`.isnull()`) in each column of 
#combined_inventory as well as the percentages (using `.mean()` instead of `.sum()`).
#Order them (increasing or decreasing) using sort_values
#Call `pd.concat` to present these in a single table (DataFrame) with the helpful column names 'count' and '%'
missing = pd.concat([listings_ordered.isnull().sum(), round(100 * listings_ordered.isnull().mean(), 2)], axis=1)
missing.columns=["Count", "%"]
missing.sort_values('Count', inplace=True, ascending=False)
missing.head(15)

,Count,%
baths,6,0.86
beds,5,0.72
taxes,5,0.72
sqft,3,0.43
district,3,0.43
elementarySchool,3,0.43
middleSchool,3,0.43
highSchool,3,0.43
address,0,0.00
city,0,0.00


Let's look at those records with missing data. 

In [29]:
# Filter on 'beds' field missing
listings_ordered[listings_ordered['baths'].isnull()]

,address,city,zip,beds,baths,sqft,listPrice,taxes,hoaFees,hoaFreq,lat,lng,district,elementarySchool,middleSchool,highSchool
66,4593 Parfet Street,Wheat Ridge,80033,NaN,NaN,"1,152",699999,"$2,822",$0,N/A,39.780613,-105.122053,Jefferson County R-1,Kullerstrand,Everitt,Wheat Ridge
153,2215 S Sheridan Boulevard,Denver,80227,NaN,NaN,NaN,649000,NaN,$0,N/A,39.675443,-105.053620,NaN,NaN,NaN,NaN
265,5475 Espana Court,Denver,80249,NaN,NaN,"2,864",585000,"$6,992",$0,N/A,39.793697,-104.757826,Denver 1,Waller,McGlone,DSST: Green Valley Ranch
588,6595 N Federal Boulevard,Denver,80221,NaN,NaN,NaN,429000,NaN,$0,N/A,39.816303,-105.025476,NaN,NaN,NaN,NaN
605,960 S Perry Street,Denver,80219,2.0,NaN,"1,081",420000,"$1,417",$0,N/A,39.699520,-105.039990,Denver 1,Castro,West Leadership,Abraham Lincoln
696,2199 W Radcliff Avenue Unit G,Englewood,80110,NaN,NaN,NaN,220000,NaN,$0,N/A,39.637269,-105.015003,NaN,NaN,NaN,NaN


Remove those records. We can't figure out the likely rent without the number of beds/baths. 

In [30]:
listings_ordered = listings_ordered[~listings_ordered['baths'].isnull()]
listings_ordered.shape

(691, 16)

### 1.6.5 Save the Homes For Sale data locally

In [31]:
listings_ordered.to_csv('../data/denverlistings.csv')